# Crypto Exchange Analysis

This notebook reads 4 different tables generated by the data pipelines on Airflow and performs some analysis to extract insights.

**Note**: Operations will be filtering a specific date partition for sample purposes, the date can be changed on the code if desired.

In [55]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', '{:,.2f}'.format)

## Collecting each table

Data is partitioned by ingestion date used on the date column, here the whole data will be read so the pandas dataframe will contain all data generated throught the time the pipeline was active

In [56]:
df_exchanges = pd.read_parquet('/home/jovyan/data/crypto_ingestion/processed/exchanges/')
df_shared_markets = pd.read_parquet('/home/jovyan/data/crypto_ingestion/processed/shared_markets/')
df_market_30day_volume =  pd.read_parquet('/home/jovyan/data/crypto_ingestion/processed/market_30day_volume/')
df_exchange_30day_volume =  pd.read_parquet('/home/jovyan/data/crypto_ingestion/processed/exchange_30day_volume/')

In [64]:
df_exchanges.head()

,exchange_id,exchange_name,year_established,country,trust_score,trust_score_rank,date
0,binance,Binance,2017,Cayman Islands,10,1,2025-08-10
1,gate,Gate,2013,Panama,10,2,2025-08-10
2,bitget,Bitget,2018,Seychelles,10,3,2025-08-10
3,bybit_spot,Bybit,2018,British Virgin Islands,10,4,2025-08-10
4,mxc,MEXC,2018,Seychelles,10,5,2025-08-10


In [65]:
df_shared_markets.head()

,exchange_id,market_id,base,target,name,date
0,okex,ETH_USD,ETH,USD,ETH/USD,2025-08-10
1,okex,BTC_USD,BTC,USD,BTC/USD,2025-08-10
2,okex,XRP_USD,XRP,USD,XRP/USD,2025-08-10
3,gdax,BTC_USD,BTC,USD,BTC/USD,2025-08-10
4,gdax,XRP_USD,XRP,USD,XRP/USD,2025-08-10


In [66]:
df_market_30day_volume.head()

,market_id,timestamp,volume,date
0,BTC_USD,2025-07-13 00:00:00,"24,351,256,098.39",2025-08-10
1,BTC_USD,2025-07-14 00:00:00,"27,310,851,454.98",2025-08-10
2,BTC_USD,2025-07-15 00:00:00,"68,915,956,378.80",2025-08-10
3,BTC_USD,2025-07-16 00:00:00,"80,299,520,980.23",2025-08-10
4,BTC_USD,2025-07-17 00:00:00,"50,374,263,848.76",2025-08-10


In [67]:
df_exchange_30day_volume.head()

,exchange_id,timestamp,volume,date
0,binance,2025-07-13 01:50:00,"129,895.45",2025-08-10
1,binance,2025-07-14 01:50:00,"135,591.33",2025-08-10
2,binance,2025-07-15 01:50:00,"265,128.26",2025-08-10
3,binance,2025-07-16 01:50:00,"257,027.18",2025-08-10
4,binance,2025-07-17 01:50:00,"249,427.92",2025-08-10


# Performin Analysis

## Top 10 exchanges by BTC Volume on 2025-08-10

In [81]:
df_exchange_30day_volume_filtered = df_exchange_30day_volume[df_exchange_30day_volume['date'] == '2025-08-10']
total_volume = df_exchange_30day_volume.groupby('exchange_id')['volume'].sum().reset_index()
top_exchanges = total_volume.nlargest(50, 'volume')

top_exchanges.head(10)

,exchange_id,volume
2,binance,"6,186,263.40"
26,gate,"1,271,532.95"
39,mxc,"1,256,322.24"
8,bitget,"1,171,773.17"
18,bybit_spot,"1,123,147.34"
21,coinw,"1,081,330.25"
16,bitvenus_spot,"1,074,674.22"
34,kcex,"999,434.82"
46,upbit,"966,277.46"
37,lbank,"949,405.10"


## Trust Score vs BTC Volume vs Year Established 30 day on 2025-08-10

In [80]:
df_exchange_30day_volume_filtered = df_exchange_30day_volume[df_exchange_30day_volume['date'] == '2025-08-10']
df_exchanges_filtered = df_exchanges[df_exchanges['date'] == '2025-08-10']

df_volume_exchange_merged = df_exchange_30day_volume_filtered.merge(df_exchanges_filtered, on="exchange_id")
df_volume_exchange_merged_grouped = df_volume_exchange_merged.groupby(["exchange_name", "trust_score", "year_established"], as_index=False)["volume"].sum()
df_volume_exchange_merged_grouped.sort_values(by=["trust_score", "volume", "year_established"], ascending=[False, False, True]).head(10)


,exchange_name,trust_score,year_established,volume
3,Binance,10,2017,"6,186,263.40"
26,Gate,10,2013,"1,271,532.95"
38,MEXC,10,2018,"1,256,322.24"
10,Bitget,10,2018,"1,171,773.17"
18,Bybit,10,2018,"1,123,147.34"
40,OKX,10,2017,"944,511.09"
21,Coinbase Exchange,10,2012,"899,838.71"
34,Kraken,10,2011,"382,927.88"
28,HTX,9,2013,"836,225.12"
23,Crypto.com Exchange,9,2019,"828,715.93"


### Most Common Shared Markets Among Top 30 Exchanges on 2025-08-10

In [111]:
df_shared_markets_filtered = df_shared_markets[df_shared_markets['date'] == '2025-08-10']
df_exchange_30day_volume_filtered = df_exchange_30day_volume[df_exchange_30day_volume['date'] == '2025-08-10']
df_exchanges_filtered = df_exchanges[df_exchanges['date'] == '2025-08-10']

df_btc_volume_agg = df_exchange_30day_volume_filtered.groupby("exchange_id")["volume"].sum().reset_index(name="total_btc_volume_30d")
top_exchanges = df_btc_volume_agg.nlargest(30, "total_btc_volume_30d")

df_top_shared_markets = top_exchanges.merge(df_shared_markets_filtered, on="exchange_id")

df_common_markets = (
    df_top_shared_markets.groupby(["base", "target"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

df_common_markets

,base,target,count
0,BTC,USD,5
1,ETH,USD,4
2,XRP,USD,4
